In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
dataset = "Datasets"
train = tf.keras.preprocessing.image_dataset_from_directory(
    f'{dataset}/train',
    seed=32,
    label_mode='categorical',
    image_size= IMAGE_SIZE,
    batch_size=BATCH_SIZE)
val = tf.keras.preprocessing.image_dataset_from_directory(
    f'{dataset}/valid',
    seed=32,
    label_mode='categorical',
    image_size= IMAGE_SIZE,
    batch_size=BATCH_SIZE)

Found 219 files belonging to 2 classes.


Found 50 files belonging to 2 classes.


In [5]:
print(train.class_names)

['bee', 'not bee']


In [6]:
model = Sequential()
resnet = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),pooling='avg',classes=2)
for layer in resnet.layers:
    layer.trainable=False
model.add(resnet)
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(2,activation='softmax'))

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(train,validation_data=val,epochs=EPOCHS)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.5937 - loss: 1.5993 - val_accuracy: 0.6800 - val_loss: 0.9767
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8578 - loss: 0.3637 - val_accuracy: 0.8000 - val_loss: 0.4544
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9161 - loss: 0.1815 - val_accuracy: 0.8400 - val_loss: 0.3367
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9817 - loss: 0.0798 - val_accuracy: 0.7600 - val_loss: 0.7501
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9837 - loss: 0.0515 - val_accuracy: 0.8600 - val_loss: 0.3840
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9905 - loss: 0.0479 - val_accuracy: 0.8200 - val_loss: 0.5378
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9839 - loss: 0.0380 - val_accuracy: 0.8200 - val_loss: 0.5817
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9989 - loss: 0.0127 - val_accuracy: 0.9000 - val_loss: 0.3200
Epoch 9/

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,738,056 (102.00 MB)

 Trainable params: 1,050,114 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 2,100,230 (8.01 MB)

In [10]:
import cv2
import os
from sklearn.metrics import accuracy_score
predict = [] 
label = []
test_dir="Datasets/test/"
for class_name in os.listdir(test_dir):
    class_path = os.path.join(test_dir, class_name)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img=cv2.imread(img_path)
            rimg= cv2.resize(img,IMAGE_SIZE)
            img= np.expand_dims(rimg,axis=0)
            print(img.shape)
            pred= model.predict(img)
            print(pred)
            print(f'This is {train.class_names[np.argmax(pred)]}')
            predict.append(class_name)
            label.append(train.class_names[np.argmax(pred)])
accuracy = accuracy_score(predict, label)
print(f'Accuracy: {accuracy * 100:.2f}%')

(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.9653423  0.03465776]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
[[0.8796566  0.12034331]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
[[9.994197e-01 5.802758e-04]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
[[0.20540461 0.7945954 ]]
This is not bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
[[2.2890682e-04 9.9977106e-01]]
This is not bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
[[9.999362e-01 6.383230e-05]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
[[0.8778044  0.12219559]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
[[0.97838986 0.02161019]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
[[0.3117709  0.68822914]]
This is not bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
[[0.01151608 0.98848397]]
This is not bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [11]:
img=cv2.imread('Datasets/test/not bee/mimicbee00018_jpeg.rf.ecfe6648e124eee71ef00fef7237b21a.jpg')
rimg= cv2.resize(img,IMAGE_SIZE)
img= np.expand_dims(rimg,axis=0)
print(img.shape)
pred= model.predict(img)
print(pred)
print(f'This is {train.class_names[np.argmax(pred)]}')

(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
[[0.04115139 0.9588486 ]]
This is not bee
